# Extract Daily Heat Index Values for Point Locations

To extract daily heat index values for point locations, we can use the exact same code we used for the national, regional, and district boundaries of Ghana. The only difference is that we are using points, not polygons.

In [ ]:
# =============================================================================
# SETUP CELL - Run this first before importing packages
# =============================================================================

"""
This cell installs geospatial packages not included in Google Colab by default.
These are essential for working with climate and satellite data.

Packages being installed:
- rasterio: Reading and writing raster data (satellite images, climate grids)
- rasterstats: Computing statistics from raster data within vector boundaries
"""
!pip install rasterio rasterstats

In [ ]:
# Dependencies
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import glob
from rasterstats import zonal_stats, gen_zonal_stats
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import time

# Get Daily Heat Index Files

First we need to get the location of our daily maximum heat index files and create a list of these files.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the base directory
base_dir = 'drive/MyDrive/Climate-Health-Data-Science-Workshop/Day3/'

In [ ]:
# Let's look at the data files available using glob
# Construct the file path using os.path.join to ensure compatibility
file_path = os.path.join(base_dir, 'data', 'himax-2016', '*.tif')

files = sorted(glob.glob(file_path))


files[:5]

## Load the DHS Clusters

Recall the DHS survey data we have used in the previous tutorials, and it is at household level, which means that there is multi-records for a given survey cluster, sharing the same long and lat. <br> For the heat value extractions, since we are looking at the cluster level,  survey data has been aggregated to DHS cluster points. All the numercial columns have been aggregated by mean(), categorical columns were mostly discarded except 'rural_urban' and 'country'    

In [ ]:
# First let's load the Ghana DHS survey dataset
df_fn = os.path.join(base_dir, 'data/DHS_Clusters_Points_Ghana_2014.csv')

df = pd.read_csv(df_fn)

# Identify records where both longitude and latitude are 0
index_to_drop = df[(df['long'] == 0) & (df['lat'] == 0)].index

# Drop these records using the drop function
df = df.drop(index_to_drop)
df.head(1)

In [ ]:
# we can check how many DHS Clusters have been sureveyed
len(df)

Adding Identifiers: This cell adds a new column to the DataFrame, assigning a unique ID to each cluster, starting from 1 up to the number of records. This is useful because later after extracting the heat we may want to join the data back

In [ ]:
# we will create a new column named 'cluster_id'
df['cluster_id'] = list(range(1, len(df)+1))

In [ ]:
df.head(5)

## The DHS data is a CSV file so it needs to be converted into a GeoPandas DataFrame with geometry, not lat/long.

In [ ]:
# Import Point
from shapely.geometry import Point

# Create geometry (vector point) from the lon and lat coordinates
geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]

# Convert pandas dataframe to geopandas dataframe
geo_df = gpd.GeoDataFrame(df, geometry=geometry)

# Set the CRS to WGS 84 (EPSG:4326)
# This step is important as later you may want to reproject the vector files
geo_df.set_crs("EPSG:4326", inplace=True)

geo_df.head(1)

## Buffer
The DHS cluster locations are randomized by up to 10 km. Thus we do not know exactly where a given survey cluster is located. To extract weather and climate data, we need to buffer the cluster by 10 km to increase the accuracy of our extraction.   

But our `crs` is in decimal degrees so we need to convert that unit to km. Below is a short function to do just this. Notice that we buffer the points, they get converted to polygons (e.g. circles).

In [ ]:
def km_to_d(km):
    """ Roughly convert Km to degrees: http://wiki.gis.com/wiki/index.php/Decimal_degrees.

    args:
        km = km distance
    """

    d = 0.01 * km / 1.11

    return d

In [ ]:
# Buffer our points
geo_df['geometry'] = geo_df['geometry'].buffer(km_to_d(5)) # ~5km buffer


## Plot the DHS household Cluster Points over the Himax map
Before we run the zonal statistics, it will be good to plot the distribution of DHS Cluster Poitns over Himax layer.

In [ ]:
# first let's read the 1st July 2016 Himax layer
fn = files[182]
himax = rasterio.open(fn)

In [ ]:
# Initialize the figure and axes
fig, ax = plt.subplots(figsize=(7, 5))

# Plot the raster using rasterio's show function
img = show(himax, ax=ax, cmap='YlOrRd')

# Plotting the point data on the same axes
geo_df.plot(ax=ax, marker='o', color='black', markersize=3)  # Customize marker style as needed

plt.tight_layout()

## Calculating Zonal Statistics
From `rasterstats`, we can use the [`zonal_stats`](https://pythonhosted.org/rasterstats/manual.html) function to build our own defined function `zonal` to easily compute the average heat index per geographic area we want. This function will encapsulate all the necessary steps required to perform this computation and can be reused for any raster or vector data input with different statistics.


In [ ]:
def zonal(rst_in, polys_in, do_stats):
    """Function will run zonal stats on a raster and a set of polygons. All touched is set to True by default.

    Args:
        rst_in (str): File name/path of raster to run zonal stats on.
        polys_in (GeoDataFrame): GeoDataFrame of polygons (e.g., administrative boundaries).
        do_stats (str): Stats to compute, see rasterstats package for documentation (e.g., 'mean', 'sum').

    Returns:
        GeoDataFrame: A GeoDataFrame with the results of the zonal statistics.
    """

    # Ensure the polygons are in the same CRS as the raster
    # Note: It's good practice to read the raster's CRS dynamically rather than hardcoding it
    with rasterio.open(rst_in) as src:
        raster_crs = src.crs

    polys_in = polys_in.to_crs(raster_crs)

    # Run Zonal Stats
    zs_feats = zonal_stats(polys_in, rst_in, stats=do_stats, geojson_out=True, all_touched=True)

    # Turn into GeoDataFrame and set CRS
    zgdf = gpd.GeoDataFrame.from_features(zs_feats)
    zgdf.crs = polys_in.crs  # Ensuring the resulting GeoDataFrame retains the original CRS

    return zgdf

Now, let's calculate the daily maximum heat index for **July 1 2016** for each buffered DHS cluster.

In [ ]:
# Load your vector data
polys_in = geo_df[['cluster_id', 'geometry']]
polys_in

In [ ]:
# Specify the raster file and the statistic you want to compute
rst_in = files[182]

do_stats = 'mean'  # This could be 'sum', 'mean', 'max', etc.

# Run the zonal statistics function
dhs = zonal(rst_in, polys_in, do_stats)

In [ ]:
dhs.head()

In [ ]:
# Plot it

# Initialize empty figure
fig = plt.figure(figsize=(10, 15))

# Add four axes
ax1 = fig.add_subplot(2,2,1)
ax2 = fig.add_subplot(2,2,2)

# DHS plots
dhs.plot('mean', legend = True, cmap = 'Reds', ax = ax1)

# Plotting the raster data
data = rasterio.open(files[182]).read(1)
# Mask NoData values (-9999)
masked_data = np.ma.masked_where(data == -9999, data)
ax2.imshow(masked_data, cmap = 'Reds')


## Now Let's Estimate Heat Index Max for Seven Days with a For Loop

In [ ]:
files[:7]

As Windows and Mac use different symbols to sepricify file path, thus the file path may also appear differently if running the jupyter notebook on different operating system. Windows use double backslash `'\\'` to divide file path, while Mac system uses forwardslash `'/'`.  If you are running this code on a Mac system, within the for loop, on code  `date = file.split('\\himax.')[1].split('-')[0] `, please CHANGE the `'\\himax.'` to `'/himax.'` to make the code run.

In [ ]:
# Now write a for loop:
# We are iterating through the first seven days of 2016

for i, file in enumerate(files[:7]):

    # Logging
    print(i, file)

    # get the date string
    date = file.split('/himax.')[1].split('-')[0]
    print(i, 'my date is', date, '\n')

    # Run the zonal statistics function
    gdf = zonal(file, polys_in, do_stats = 'mean')

    # Updates the column name from a generic 'mean' to the specific date
    gdf.rename(columns = {'mean' : date}, inplace = True)

    # Make a copy of the first date
    if i == 0:

        gdf_out = gdf.copy()

    # Merge days 2-10 onto day 1
    else:
        gdf_out = pd.merge(gdf_out, gdf[['cluster_id', date]], on = 'cluster_id', how = 'inner')

If you check the output file `gdf_out`, you can see that the date columns have been added, and they represent the average himax value within the ~5km buffer of the corresponding DHS cluster

In [ ]:
# check the output file
gdf_out

# Visualizing time series data
The following code plot the heat index values over sseven days for a particular DHS cluster from the dataset.

In [ ]:
# substract the second row - which is Cluster 2
y = gdf_out.iloc[1,2:].values

In [ ]:
# Converts the date columns (starting from the 3rd column) into pandas datetime objects.
x = pd.to_datetime(gdf_out.columns.values[2:])

In [ ]:
plt.scatter(x,y);
plt.plot(x,y)
plt.title('Heat Index for DHS Cluster 2 for Jan 1 - 7 2016');
plt.xticks(rotation=90)
#plt.xlabel('Date')
plt.ylabel('Daily Maximum Heat Index °C')
plt.ylim([0,45])
plt.show()